In [2]:
import pandas
from os import listdir

def getFiles(s):
    return map(lambda x: f"{s}/{x}", listdir(s))
# filenames = [*getFiles("out-results"), *getFiles("out-shuffle"), *getFiles("out-structure-10-10")]
filenames = getFiles("out")
files = [pandas.read_csv(file, sep=';')
         for file in filenames if 'scores-' not in file]

df = pandas.concat(files)

# rest_filenames = getFiles("out-rerun")
# rest_files = [pandas.read_csv(file, sep=';')
#          for file in rest_filenames if 'scores-' not in file]
# df2 = pandas.concat(rest_files)
# df2 = df2.loc[~df2['Algorithm'].isin(['StructuredUnification', 'StructuredSplitUnification', 'SplitTypeUnification', 'SplitTypeSplitUnification', 'SimpleUnification', 'SimpleSplitUnification'])]

In [3]:
combined = df
trimmed = combined[['Algorithm', 'Ranking Factor', 'Size Factor', 'Index', 'Line', 'Character', 'Keywords Before']]
trimmed.loc[trimmed['Ranking Factor'].isnull(), 'Ranking Factor'] = 0
trimmed.loc[trimmed['Size Factor'].isnull(), 'Size Factor'] = 0
groups = trimmed.groupby(['Algorithm', 'Ranking Factor', 'Size Factor'])

In [26]:
fullDump = True
prefix0 = False
applyrewrite = True

In [48]:
from collections import defaultdict as dd
from itertools import product
allresults: dict[tuple, dict[int, tuple[int, dict[int, int]]]] = {}  # groupname -> lemmaPosition -> (count, (resultIndex -> count))
data = []
allWords = set()
allCounts = set()
for (groupName, frame), keyword in product(groups, ['apply', 'rewrite'] if applyrewrite else [None]):
    # if 'Simple' not in groupName[0]: continue
    if not fullDump and groupName[1] not in [0,5]: continue
    if not fullDump and groupName[2] not in [0,5]: continue
    # if (not (
    #     "Structure" in groupName[0]
    #     and groupName[1] == 2
    #     and groupName[2] == 5)): continue
    line = -1
    ch = -1
    lemmaPosition = -1
    thisdata = dd(lambda: dd(int))
    positionCounts = dd(int)
    allresults[groupName] = {}
    for resultIndex, l, c, kw in frame[['Index', 'Line', 'Character', 'Keywords Before']].values:
        allWords.add(kw)
        if resultIndex == -2: continue
        # if (type(kw) != str): continue
        if line != l:
            line = l
            ch = c
            lemmaPosition = 0
        if ch != c:
            if ch + 1 == c:
                lemmaPosition += 1
            else:
                lemmaPosition = 0
            ch = c
        if not keyword or (type(kw) == str and keyword in kw):
            thisdata[lemmaPosition][resultIndex] += 1
            positionCounts[lemmaPosition] += 1
    
    grades = dd(int)
    for lemmaPosition, results in thisdata.items():
        attempts = positionCounts[lemmaPosition]
        allCounts.add(attempts)
        allresults[groupName].update({lemmaPosition: (attempts, results)})
        grades[lemmaPosition] = sum(map(lambda j: results[j] / (j + 1), range(10))) / attempts
    a = sum(map(lambda x: grades[x] / (x + 1), range(1 if prefix0 else 9)))
    b = sum(map(lambda x: 1 / (x+1), range(1 if prefix0 else 9)))
    if b == 0:
        print(groupName, a, "failed")
        continue
    score = 100 * ( a / b )
    data.append((*groupName, *([keyword] if applyrewrite else []), round(score, 2)))
result_df = pandas.DataFrame(data, columns=['Algorithm', 'Atomic Bias', 'Size Bias', *(['Keyword'] if applyrewrite else []), 'Score'])
# result_df.sort_values(by=['Score'], inplace=True, ascending=False)
print(result_df.to_clipboard(index=False))

None


In [49]:
print(allCounts)

{2, 3, 4, 133, 1542, 7, 5, 9, 394, 907, 12, 13, 782, 143, 272, 16, 15, 19, 1172, 781, 22, 27, 28, 30, 6, 160, 416, 418, 162, 415, 38, 39, 296, 295, 43, 45, 47, 51, 10, 55, 11, 393, 64, 321, 706, 707, 197, 454, 70, 73, 81, 83, 854, 344, 345, 219, 987, 1628, 1118, 221, 93, 1377, 220, 739, 1254, 487, 123, 364, 112, 498, 499, 634, 122, 635, 382, 383}


In [6]:
result_df.to_latex('results.tex', index=False)

/tmp/ipykernel_20870/259440309.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  result_df.to_latex('results.tex', index=False)


In [12]:
good_names = {
'Basic': 'Baseline',
'Shuffle': 'Shuffle',
'SimpleTypeIntersection': 'Type Intersection',
'SimpleUnification': 'Type Intersection with Unification',
'SimpleSplitUnification': 'Type Intersection with Split Unification',
'SplitTypeIntersection': 'Split Type Intersection',
'SplitTypeUnification': 'Split Type Intersection with Unification',
'SplitTypeSplitUnification': 'Split Type Intersection with Split Unification',
'StructuredTypeEvaluation': 'Type Structure',
'StructuredUnification': 'Type Structure with Unification',
'StructuredSplitUnification': 'Type Structure with Split Unification',
'ShuffleSplitUnification': 'Shuffle with Split Unification',
'ShuffleUnification': 'Shuffle with Unification',
}

In [13]:
tableLength = 8
foreach = {}
for (algo, rank, size), v in allresults.items():
    lmao = []
    foreach[(algo, rank, size)] = lmao
    for lemmaPosition, (amount, results) in v.items():
        if lemmaPosition > tableLength:
            break
        first = results[0]
        second = results[1]
        third = results[2]
        rest = sum(map(lambda x: results[x], range(3, 10)))
        fail = amount - sum([first, second, third, rest])
        passed = amount - fail
        lmao.append([*map(lambda x: round(x * 100 / amount, 2), [first, second, third, rest, passed, fail])])
    print(r'\begin{table}[h]')
    print(r'\footnotesize')
    print('\\begin{tabular}{|r|', *map(lambda x: "r", range(tableLength + 1)), r'|}\hline', sep="")
    # print(f'\multicolumn{{{tableLength + 2}}}{{l}}{{\\normalsize', k, '}', '\\\\', '\\hline')
    print("Prefix", *map(lambda x: rf'\ctd{{{x}}}', range(tableLength)), rf'\ctdl{{{tableLength}}}', sep=' & ', end='\\\\\\hline\n')
    for x, n in enumerate(["1st","2nd","3rd","4-10th","1-10th","Fail"]):
        print(n, *map(lambda l: f'{l[x]:.1f}\\%', lmao), sep=' & ', end=r"\\")
        if x == 3:
            print(r"\hline", end="")
        print()
    print('\\hline\\end{tabular}')
    print(r'\vspace{0.4em}')
    params = '' if rank == 0 else f', with Atomic Bias {int(rank)} and Size Bias {int(size)}'
    print(rf'\caption{{Test results from {good_names[algo]}{params}, showing for each prefix length how large a proportion of results appeared at each position in the completions.}}')
    print(r'\label{tab:appendix-results-', algo.lower(), ('-' + str(int(rank)) + '-' + str(int(size)) if rank else ''), '}', sep='')
    print(r'\end{table}')
    print()

\begin{table}[h]
\footnotesize
\begin{tabular}{|r|rrrrrrrrr|}\hline
Prefix & \ctd{0} & \ctd{1} & \ctd{2} & \ctd{3} & \ctd{4} & \ctd{5} & \ctd{6} & \ctd{7} & \ctdl{8}\\\hline
1st & 0.3\% & 11.4\% & 32.7\% & 36.9\% & 40.0\% & 43.9\% & 49.6\% & 52.0\% & 55.4\%\\
2nd & 0.2\% & 10.5\% & 12.2\% & 9.9\% & 8.5\% & 11.3\% & 9.6\% & 11.3\% & 10.8\%\\
3rd & 0.3\% & 6.4\% & 6.8\% & 7.2\% & 8.6\% & 6.7\% & 7.3\% & 6.5\% & 5.5\%\\
4-10th & 9.3\% & 20.0\% & 18.1\% & 18.0\% & 17.3\% & 15.7\% & 13.0\% & 11.8\% & 11.6\%\\\hline
1-10th & 10.1\% & 48.3\% & 69.8\% & 72.1\% & 74.4\% & 77.6\% & 79.5\% & 81.5\% & 83.3\%\\
Fail & 89.9\% & 51.7\% & 30.2\% & 27.9\% & 25.6\% & 22.4\% & 20.5\% & 18.5\% & 16.7\%\\
\hline\end{tabular}
\vspace{0.4em}
\caption{Test results from Baseline, showing for each prefix length how large a proportion of results appeared at each position in the completions.}
\label{tab:appendix-results-basic}
\end{table}

\begin{table}[h]
\footnotesize
\begin{tabular}{|r|rrrrrrrrr|}\hline
Prefix

In [21]:
d = []
for r55, r25 in zip(foreach[('StructuredSplitUnification', 5.0, 5.0)], foreach[('StructuredTypeEvaluation', 5.0, 5.0)]):
    d.append([*map(lambda x: x[0] - x[1], zip(r55, r25))])
for x, n in enumerate(["1st","2nd","3rd","4-10th","1-10th","Fail"]):
    print(n, *map(lambda l: f'{l[x]:+.1f}', d), sep=' & ', end=r"\\")
    if x == 3:
        print(r"\hline", end="")
    print()

1st & +0.8 & +0.4 & +0.1 & -0.0 & +0.0 & -0.0 & -0.0 & -0.0 & -0.0\\
2nd & -0.2 & -0.4 & -0.1 & +0.1 & +0.0 & +0.0 & +0.0 & +0.0 & +0.0\\
3rd & -0.2 & +0.1 & +0.1 & +0.1 & +0.0 & +0.0 & +0.0 & +0.0 & +0.1\\
4-10th & -0.3 & -0.0 & -0.0 & -0.2 & +0.0 & +0.1 & +0.0 & +0.0 & -0.1\\\hline
1-10th & +0.1 & +0.0 & +0.0 & +0.0 & +0.0 & +0.0 & -0.0 & -0.0 & -0.0\\
Fail & -0.1 & -0.0 & -0.0 & -0.0 & -0.1 & -0.0 & +0.0 & +0.0 & +0.0\\


In [47]:
# matrix = [
#     ['Basic', None, None],
#     ['Shuffle', 'ShuffleUnification', 'ShuffleSplitUnification'],
#     ['SimpleTypeIntersection', 'SimpleUnification', 'SimpleSplitUnification'],
#     ['SplitTypeIntersection', 'SplitTypeUnification', 'SplitTypeSplitUnification'],
#     ['StructuredTypeEvaluation', 'StructuredUnification', 'StructuredSplitUnification'],
# ]
matrix = [
    ['Basic', None],
    ['Shuffle', 'ShuffleSplitUnification'],
    ['SimpleTypeIntersection', 'SimpleSplitUnification'],
    ['SplitTypeIntersection', 'SplitTypeSplitUnification'],
    ['StructuredTypeEvaluation', 'StructuredSplitUnification'],
]
print(r'\begin{table}[h]')
print(r"""\begin{tabular}{|r|l|p{2cm}|p{2cm}|}\hline
Algorithm & apply & rewrite & apply, Split Unification & rewrite, Split Unification \\\hline""")
for row in matrix:
    print(good_names[row[0]], end=' ')
    for algo in row:
        if algo is None:
            print('&   -  ', end=' ')
            continue
        filt = result_df.loc[result_df['Algorithm'] == algo, ['Algorithm','Keyword', 'Score']]
        print('&', filt.loc[result_df['Keyword'] == 'apply', ['Algorithm','Keyword', 'Score']].values[0], end=' ')
        print('&', filt.loc[result_df['Keyword'] == 'rewrite', ['Algorithm','Keyword', 'Score']].values[0], end=' ')
    print(r'\\')
print(r'\hline\end{tabular}')
print(r'\vspace{0.4em}')
print(r'\caption{Summary of the experiment results. Parameters for Type Structure are a ranking factor of 5 and size factor of 5 in all three cases, as it performed the best.}')
print(r'\label{tab:score-summary}')
print(r'\end{table}')

\begin{table}[h]
\begin{tabular}{|r|l|p{2cm}|p{2cm}|}\hline
Algorithm & apply & rewrite  & apply, Split Unification & rewrite, Split Unification \\\hline
Baseline & ['Basic' 'apply' 30.94] & ['Basic' 'rewrite' 24.87] &   -   \\
Shuffle & ['Shuffle' 'apply' 26.69] & ['Shuffle' 'rewrite' 24.22] & ['ShuffleSplitUnification' 'apply' 27.11] & ['ShuffleSplitUnification' 'rewrite' 24.26] \\
Type Intersection & ['SimpleTypeIntersection' 'apply' 50.99] & ['SimpleTypeIntersection' 'rewrite' 46.34] & ['SimpleSplitUnification' 'apply' 51.39] & ['SimpleSplitUnification' 'rewrite' 46.41] \\
Split Type Intersection & ['SplitTypeIntersection' 'apply' 52.03] & ['SplitTypeIntersection' 'rewrite' 46.42] & ['SplitTypeSplitUnification' 'apply' 52.33] & ['SplitTypeSplitUnification' 'rewrite' 46.44] \\
Type Structure & ['StructuredTypeEvaluation' 'apply' 43.26] & ['StructuredTypeEvaluation' 'rewrite' 24.11] & ['StructuredSplitUnification' 'apply' 43.41] & ['StructuredSplitUnification' 'rewrite' 24.15] \\
\hl

In [46]:
def getFiles(s):
    return map(lambda x: f"{s}/{x}", listdir(s))
filenames = getFiles("out")
time_files = [pandas.read_csv(file, sep=';')
         for file in filenames if 'scores' in file]
time_df = pandas.concat(time_files)
time_df = time_df[['Algorithm', 'Ranking Factor', 'Size Factor', 'Time']].groupby(['Algorithm', 'Ranking Factor', 'Size Factor']).mean()
time_df

,,,Time
Algorithm,Ranking Factor,Size Factor,
Basic,0,0,28834.164021
Shuffle,0,0,29312.074074
ShuffleSplitUnification,0,0,29366.619048
ShuffleUnification,0,0,29349.248677
SimpleSplitUnification,0,0,29880.746032
SimpleTypeIntersection,0,0,30277.021164
SimpleUnification,0,0,29818.714286
SplitTypeIntersection,0,0,30558.571429
SplitTypeSplitUnification,0,0,30265.365079


In [22]:
x = result_df[result_df['Algorithm'] == 'StructuredSplitUnification']
ressss = [
    [0,0,0,0],
    [0,0,0,0],
    [0,0,0,0],
    [0,0,0,0],
]
inds = [1,2,5,10]
for algo, rf, sf, s in x.values:
    ressss[inds.index(sf)][inds.index(rf)] = s
print('     ', *inds[:3], sep='   &   ', end=r'   \\\hline')
print()
for i in range(3):
    print('~~', inds[i], '~~', end='')
    for j in range(3):
        print(' &', f'{ressss[i][j]:.2f}', end='')
    print(r' \\')

        &   1   &   2   &   5   \\\hline
~~ 1 ~~ & 37.48 & 45.71 & 48.68 \\
~~ 2 ~~ & 46.76 & 50.13 & 50.76 \\
~~ 5 ~~ & 50.66 & 51.06 & 51.13 \\
